In [4]:
import pandas as pd

# Load your entire Excel data without limiting the number of rows
file_path = 'Todays Python Workflow.xlsx'  # Replace with the correct path
df = pd.read_excel(file_path, usecols="A:J")  # Load all rows without specifying nrows

# Column K: Strike
df['Strike'] = df['Option'].str.extract(r'(\d{4})')

# Column L: Expiration
df['Expiration'] = df['Option'].str.extract(r'(\d{1,2} \w{3} \d{2})')

# Column M: Type
df['Type'] = df['Option'].str[-1].map({'P': 'Put', 'C': 'Call'})

# Column N: Premium (Qty * Price * 100)
df['Premium'] = df['Qty'] * df['Price'] * 100

# Ensure Premium column doesn't have $ or commas and is float
df['Premium'] = df['Premium'].replace([None], 0).astype(float)  # Ensure Premium is not NaN and is a float

# Column O: Spread - leave blank for now
df['Spread'] = ""

# Column P: Difference - leave blank for now
df['Difference'] = ""

# Column Q: Bot/Sold
def determine_bot_sold(price, market):
    try:
        if isinstance(market, str) and 'x' in market:
            left, right = market.split('x')
            left, right = float(left), float(right)
            if price == left:
                return "Sold"
            elif price == right:
                return "Bot"
            else:
                return "Mid"
        else:
            return "Mid"  # Default to "Mid" if the market value is not valid
    except ValueError:
        return "Mid"  # Return "Mid" if the market values cannot be converted to floats

df['Bot/Sold'] = df.apply(lambda row: determine_bot_sold(row['Price'], row['Market']), axis=1)

# Column R: SPX High (moved from Column S)
df['SPX High'] = ""  # You will need to use yfinance or another source to pull this data

# Column S: SPX Low (moved from Column T)
df['SPX Low'] = ""  # Same as above for SPX Low

# Column T: Range (moved from Column U) - leave blank for now
df['Range'] = ""

# Column U: Bot Calls (moved from Column V)
df['Bot Calls'] = df.apply(lambda row: row['Premium'] if row['Bot/Sold'] == 'Bot' and row['Type'] == 'Call' else "", axis=1)

# Column V: Bot Puts (moved from Column W)
df['Bot Puts'] = df.apply(lambda row: row['Premium'] if row['Bot/Sold'] == 'Bot' and row['Type'] == 'Put' else "", axis=1)

# Column W: Sold Calls (moved from Column X)
df['Sold Calls'] = df.apply(lambda row: row['Premium'] if row['Bot/Sold'] == 'Sold' and row['Type'] == 'Call' else "", axis=1)

# Column X: Sold Puts (moved from Column Y)
df['Sold Puts'] = df.apply(lambda row: row['Premium'] if row['Bot/Sold'] == 'Sold' and row['Type'] == 'Put' else "", axis=1)

# Mid Call Premium = sum of all Premium from Calls if Type equals "Mid"
df['Mid Call Premium'] = df.apply(lambda row: row['Premium'] if row['Bot/Sold'] == 'Mid' and row['Type'] == 'Call' else "", axis=1)

# Mid Put Premium = sum of all Premium from Puts if Type equals "Mid"
df['Mid Put Premium'] = df.apply(lambda row: row['Premium'] if row['Bot/Sold'] == 'Mid' and row['Type'] == 'Put' else "", axis=1)

# Save the modified data to the existing Excel file
df.to_excel('Todays Python Workflow.xlsx', index=False)

# Save filtered workbook that excludes rows with 'Spread' under 'Conditions'
filtered_df = df[df['Condition'] != 'Spread']
filtered_df.to_excel('NoSpread.xlsx', index=False)

# Now, we will implement the new function to save a separate workbook with time-based worksheets

def save_hourly_workbook(df):
    # Extract the hour from the Time column (which contains datetime.time)
    df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour
    
    # Use 'with' statement to handle the ExcelWriter context, ensuring it closes automatically
    with pd.ExcelWriter('Hourly_Workbook.xlsx', engine='xlsxwriter') as writer:
        # Define the time slots for the 10 sheets, descending from "5a.m." to "2p.m."
        time_slots = ['5a.m.', '6a.m.', '7a.m.', '8a.m.', '9a.m.', '10a.m.', '11a.m.', '12p.m.', '1p.m.', '2p.m.']
        
        for i, slot in enumerate(time_slots, start=5):
            # Filter the rows corresponding to the current time slot
            hour_data = df[df['Hour'] == i]
            
            # Write the filtered data to the corresponding sheet
            hour_data.to_excel(writer, sheet_name=slot, index=False)

# Call the function to save the transactions into hourly worksheets
save_hourly_workbook(df)

